In [ ]:
# python environment stuff
IMAGED11_PATH = None  # means do not use git, otherwise "ImageD11" or "ImageD11_version_xx", etc
CHECKOUT_PATH = None  # None means guess, or you can specify a folder for the checkout

tmap_path = '/path/to/tmap.h5'

In [ ]:
if IMAGED11_PATH is not None:
    exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
    setup_ImageD11_from_git(CHECKOUT_PATH, IMAGED11_PATH)

In [ ]:
import numba
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from ImageD11.sinograms.tensor_map import TensorMap, fast_invert, ubi_and_b_to_u, ubi_and_unitcell_to_eps_crystal

%matplotlib inline

In [ ]:
tmap = TensorMap.from_h5(tmap_path)

In [ ]:
tmap.plot('ipf_x')
tmap.plot('ipf_y')
tmap.plot('ipf_z')

# B matrix recomputation

When we perform a point-by-point refinement of each voxel, we refine in two separate steps:
- The $\matrix{UB}^{-1}$ matrix of the lattice
- The strain tensor in the lab frame $\varepsilon_{\text{lab}}$

Those familiar with some crystallography will know that the $\matrix{B}$ matrix is supposed to encode the full strain tensor:

$\varepsilon_{\text{crystal}} = \frac{1}{2}\left(\matrix{B_0}\matrix{B}^{-1}+\left(\matrix{B_0}\matrix{B}^{-1}\right)^{\dagger}\right) - \matrix{I}$

where $\matrix{B_0}$ is the reference "strain-free" $\matrix{B}$ matrix for the lattice.

However, since we performed a separate refinement of the strain tensor, we want to update the $\matrix{B}$ matrix to account for the new strains.

This brings the strain from the $\matrix{UB}^{-1}$ field (via $\matrix{B}$) in line with the strain from the refinement.

# Current strain discrepancy

In [ ]:
eps_crystal_from_B_before = ubi_and_unitcell_to_eps_crystal(tmap.UBI, tmap.dzero_unitcell)

In [ ]:
fig, ax = plt.subplots()
ax.hist(tmap.eps_crystal.ravel(), bins=500, label='From eps_crystal', alpha=0.5)
ax.hist(eps_crystal_from_B_before.ravel(), bins=500, label='From UBI', alpha=0.5)
ax.set(xlabel='strain', ylabel='frequency', title='Histogram of strains')
ax.legend()
plt.show()

In [ ]:
strain_diff_before = (tmap.eps_crystal - eps_crystal_from_B_before).ravel()

fig, ax = plt.subplots()
ax.hist(strain_diff_before, bins=500)
ax.set(xlabel='strain diff', ylabel='frequency', title='Histogram of strain differences before')
plt.show()

In [ ]:
np.nanmin(strain_diff_before), np.nanmedian(strain_diff_before), np.nanmax(strain_diff_before)

# Current unitcell distribution

In [ ]:
avals_before = tmap.unitcell[0, ..., 0].ravel()

fig, ax = plt.subplots()
ax.hist(avals_before, bins=500)
ax.set(xlabel='a axis', ylabel='frequency', title='Histogram of a axis before recomputing B')
plt.show()

bvals_before = tmap.unitcell[0, ..., 1].ravel()

fig, ax = plt.subplots()
ax.hist(bvals_before, bins=500)
ax.set(xlabel='b axis', ylabel='frequency', title='Histogram of b axis before recomputing B')
plt.show()

cvals_before = tmap.unitcell[0, ..., 2].ravel()

fig, ax = plt.subplots()
ax.hist(cvals_before, bins=500)
ax.set(xlabel='c axis', ylabel='frequency', title='Histogram of c axis before recomputing B')
plt.show()

In [ ]:
print(np.nanmean(tmap.unitcell[0, ...]))

# Compute new B matrices from the strain tensors

In [ ]:
B0 = tmap.phases[0].B  # strain free B matrix from reference phase
B0

There is a neat trick for this:

$\varepsilon_{\text{crystal}} = \frac{1}{2}\left(\matrix{B_0}\matrix{B}^{-1}+\left(\matrix{B_0}\matrix{B}^{-1}\right)^{\dagger}\right) - \matrix{I}$

$\varepsilon_{\text{crystal}} + \matrix{I} = \frac{1}{2}\left(\matrix{B_0}\matrix{B}^{-1}+\left(\matrix{B_0}\matrix{B}^{-1}\right)^{\dagger}\right) $

$2\left(\varepsilon_{\text{crystal}} + \matrix{I}\right) = \matrix{B_0}\matrix{B}^{-1}+\left(\matrix{B_0}\matrix{B}^{-1}\right)^{\dagger} $

$\matrix{S} = \varepsilon_{\text{crystal}} + \matrix{I}$

$\matrix{M} = \matrix{B_0}\matrix{B}^{-1}$

$2\matrix{S} = \matrix{M}+\matrix{M}^{\dagger}$

$\matrix{B_0}$ is upper-triangular, and so is $\matrix{B}^{-1}$

Therefore $\matrix{M} = \matrix{B_0}\matrix{B}^{-1}$ is also upper-triangular:

$M = \begin{pmatrix} 
a & b & c \\
0 & d & e \\
0 & 0 & f 
\end{pmatrix}$

therefore:

$\matrix{M}+\matrix{M}^{\dagger} = \begin{pmatrix} 
2a & b & c \\
b & 2d & e \\
c & e & 2f 
\end{pmatrix}$

$\matrix{S} = \varepsilon_{\text{crystal}} + \matrix{I}$ which is symmetric:

$\matrix{S} = \begin{pmatrix} 
S_{11} & S_{12} & S_{13} \\
S_{12} & S_{22} & S_{23} \\
S_{13} & S_{23} & S_{33} 
\end{pmatrix}$

therefore:

$\matrix{S} = \begin{pmatrix} 
S_{11} & S_{12} & S_{13} \\
S_{12} & S_{22} & S_{23} \\
S_{13} & S_{23} & S_{33} 
\end{pmatrix} = \frac{1}{2}\begin{pmatrix} 
2a & b & c \\
b & 2d & e \\
c & e & 2f 
\end{pmatrix}= \begin{pmatrix} 
a & \frac{b}{2} & \frac{c}{2} \\
\frac{b}{2} & d & \frac{e}{2} \\
\frac{c}{2} & \frac{e}{2} & f 
\end{pmatrix}$

therefore:

$\matrix{M} = \begin{pmatrix} 
S_{11} & 0 & 0 \\
0 & S_{22} & 0 \\
0 & 0 & S_{33} 
\end{pmatrix} + 2\begin{pmatrix} 
0 & 2S_{12} & 2S_{13} \\
0 & 0 & 2S_{23} \\
0 & 0 & 0 
\end{pmatrix}$

which in code is:

```
M = np.diag(np.diag(S)) + 2 * np.triu(S, k=1)
```

In [ ]:
@numba.guvectorize([(numba.float64[:, :], numba.float64[:,:], numba.float64[:,:])], '(n,n),(n,n)->(n,n)', nopython=True, cache = True, target='parallel')
def epsilon_to_b(eps_crystal, B0, res):
    """
    xfab.laue.epsilon_to_b - made to go fast
    """

    if np.isnan(eps_crystal[0,0]):
        res[...] = np.nan
    else:
        S = eps_crystal + np.eye(3)
        M = np.diag(np.diag(S)) + 2 * np.triu(S, k=1)
        M_inv = np.linalg.inv(M)
        B = np.dot(M_inv, B0)
        res[...] = B

In [ ]:
B_new = epsilon_to_b(tmap.eps_crystal, B0)

In [ ]:
fig, ax = plt.subplots(layout='constrained')
im = ax.imshow(B_new[0, ..., 0, 0] - tmap.B[0, ..., 0, 0], origin='lower')
ax.set_xlabel('Lab X axis --->')
ax.set_ylabel('Lab Y axis --->')
ax.set_title('B[0,0] new - B[0,0] old')
plt.colorbar(im)
plt.show()

In [ ]:
# now we have a new B matrix, we can recompute UBI, then update the tensormap

In [ ]:
UB_new = tmap.U @ B_new

In [ ]:
UBI_new = fast_invert(UB_new)

In [ ]:
# check that the U matrix doesn't change
U_old = tmap.U.copy()

In [ ]:
# now update the UBI in the tensormap
tmap.UBI = UBI_new

In [ ]:
eps_crystal_from_B_after = ubi_and_unitcell_to_eps_crystal(tmap.UBI, tmap.dzero_unitcell)

In [ ]:
fig, ax = plt.subplots()
ax.hist(tmap.eps_crystal.ravel(), bins=500, label='From eps_crystal', alpha=0.5)
ax.hist(eps_crystal_from_B_after.ravel(), bins=500, label='From UBI', alpha=0.5)
ax.set(xlabel='strain', ylabel='frequency', title='Histogram of strains after correction')
ax.legend()
plt.show()

In [ ]:
strain_diff_after = (tmap.eps_crystal - eps_crystal_from_B_after).ravel()

fig, ax = plt.subplots()
ax.hist(strain_diff_before, bins=500, label='before')
ax.hist(strain_diff_after, bins=500, label='after')
ax.set_xlim(-3e-4, 3e-4)
ax.set(xlabel='strain diff', ylabel='frequency', title='Histogram of strain differences')
ax.legend()
plt.show()

In [ ]:
np.nanmin(strain_diff_after), np.nanmedian(strain_diff_after), np.nanmax(strain_diff_after)

In [ ]:
assert np.allclose(tmap.U, U_old, equal_nan=True)

In [ ]:
assert np.allclose(tmap.B, B_new, equal_nan=True)

In [ ]:
avals_after = tmap.unitcell[0, ..., 0].ravel()

fig, ax = plt.subplots()
ax.hist(avals_before, bins=500, alpha=0.5, label='before')
ax.hist(avals_after, bins=500, alpha=0.5, label='after')
ax.set(xlabel='a axis', ylabel='frequency', title='Histogram of a axis')
ax.legend()
plt.show()

bvals_after = tmap.unitcell[0, ..., 1].ravel()

fig, ax = plt.subplots()
ax.hist(bvals_before, bins=500, alpha=0.5, label='before')
ax.hist(bvals_after, bins=500, alpha=0.5, label='after')
ax.set(xlabel='b axis', ylabel='frequency', title='Histogram of b axis')
ax.legend()
plt.show()

cvals_after = tmap.unitcell[0, ..., 2].ravel()

fig, ax = plt.subplots()
ax.hist(cvals_before, bins=500, alpha=0.5, label='before')
ax.hist(cvals_after, bins=500, alpha=0.5, label='after')
ax.set(xlabel='c axis', ylabel='frequency', title='Histogram of c axis')
ax.legend()
plt.show()

In [ ]:
tmap.to_h5('new_tmap.h5')
tmap.to_paraview('new_tmap.h5')